In [ ]:
#Sasrec History Length Ablation

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_seed, init_logger, get_trainer, get_model
from recbole.evaluator import Evaluator
from recbole.data.dataloader import FullSortEvalDataLoader
from recbole.sampler import Sampler
import json
import os
import torch
from logging import getLogger
import numpy as np
from recbole.data.dataloader import TrainDataLoader


def filter_users_by_history_length(dataset, history_length=1):
    mask = dataset.inter_feat['item_length'] == history_length
    filtered_inter_feat = dataset.inter_feat[mask]
    filtered_dataset = dataset.copy(filtered_inter_feat)
    return filtered_dataset

def filter_data_by_valid_items(data, n_items,config):
    item_seqs = data.inter_feat[config['ITEM_SEQ_FIELD']]
    
    mask = torch.all(item_seqs < n_items, dim=1)
    
    filtered_inter_feat = data.inter_feat[mask]
    
    filtered_data = data.copy(filtered_inter_feat)
    return filtered_data
def sasrec_with_filtered_evaluation(domain, base_dir, history_length, model_file):

    # Load model
    if not os.path.exists(model_file):
        raise FileNotFoundError(f"Model file not found: {model_file}")
    
    try:
        checkpoint = torch.load(model_file)
        config = checkpoint["config"]
        init_seed(config["seed"], config["reproducibility"])
        init_logger(config)
        logger = getLogger()
        logger.info(config)
    except Exception as e:
        print(f"Error loading model configuration: {str(e)}")
        raise

    config['data_path']=base_dir
    config['dataset']=domain
    config['device'] = 'cuda'

    
    # Create dataset and prepare data
    dataset = create_dataset(config)
    logger.info(dataset)
    train_data, valid_data, test_data = data_preparation(config, dataset)
    
       # Calculate the number of items in the dataset
    n_items = len(dataset.inter_feat[config['ITEM_ID_FIELD']].unique())
    print(f"Total number of items in the dataset: {n_items}")


    # Apply filtering to train, valid, and test datasets
    filtered_train_dataset = filter_data_by_valid_items(train_data.dataset, n_items,config)
    filtered_valid_dataset = filter_data_by_valid_items(valid_data.dataset, n_items,config)
    filtered_test_dataset = filter_data_by_valid_items(test_data.dataset, n_items,config)

    # Create appropriate data loaders
    train_sampler = Sampler(config, filtered_train_dataset)
    train_data = TrainDataLoader(config, filtered_train_dataset, train_sampler, shuffle=True)
    
    valid_sampler = Sampler(config, filtered_valid_dataset)
    valid_data = FullSortEvalDataLoader(config, filtered_valid_dataset, valid_sampler)
    
    test_sampler = Sampler(config, filtered_test_dataset)
    test_data = FullSortEvalDataLoader(config, filtered_test_dataset, test_sampler)

    # Filter test dataset
    filtered_test_dataset = filter_users_by_history_length(test_data.dataset, history_length=history_length)

    sampler = Sampler(config, filtered_test_dataset)
    filtered_test_data = FullSortEvalDataLoader(config, filtered_test_dataset, sampler)

    # Load model
    try:
        model = get_model(config["model"])(config, train_data.dataset).to(config["device"])
        model.load_state_dict(checkpoint["state_dict"])
        model.load_other_parameter(checkpoint.get("other_parameter"))
    except Exception as e:
        print(f"Error loading model: {str(e)}")
        raise

    # Evaluation
    trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)
    test_result = trainer.evaluate(filtered_test_data, load_best_model=False, show_progress=config['show_progress'])

    print("Final Filtered Test Results:")
    print(test_result)

    return test_result



base_dir = '/kaggle/input/data-ablation/Archive/'
domains=['Baby_Products','Video_Games','All_Beauty']
models=['/kaggle/input/data-ablation/SAS_Baby.pth','/kaggle/input/data-ablation/SAS_Video.pth','/kaggle/input/data-ablation/SAS_Beauty.pth']
all_results = {domain: {} for domain in domains}
for i,domain in enumerate(domains):
    ncdg_10=[]
    ncdg_50=[]
    for history in range(2,9):
        results = sasrec_with_filtered_evaluation(domain, base_dir+f'{domain}', history, models[i])
        ncdg_10.append(results['ndcg@10'])
        ncdg_50.append(results['ndcg@50'])
    all_results[domain]['ndcg@10']=ncdg_10
    all_results[domain]['ndcg@50']=ncdg_50
        


In [ ]:
#Unisrec history length ablation

from logging import getLogger
import torch
from recbole.config import Config
from recbole.data import data_preparation
from recbole.utils import init_seed, init_logger, get_trainer
from utils import get_model, create_dataset
import os 
import json
from recbole.evaluator import Evaluator
from recbole.data.dataloader import FullSortEvalDataLoader
from recbole.sampler import KGSampler, Sampler, RepeatableSampler

def check_path(path):
    if not os.path.exists(path):
        os.makedirs(path)

def filter_users_by_history_length(dataset, history_length=1):
    # Get users with the specific history length
    mask = dataset.inter_feat['item_length'] == history_length
    filtered_inter_feat = dataset.inter_feat[mask]
    
    # Create a new dataset with filtered interactions
    filtered_dataset = dataset.copy(_)
    filtered_dataset.inter_feat = filtered_inter_feat
    return filtered_dataset


def run(model_name,domain,base_dir,model_file,history_length):

    props = ['overall.yaml', f'{model_name}.yaml']


    # Add checkpoint_dir to kwargs
    model_class = get_model(model_name)

    # configurations initialization
    config = Config(model=model_class, dataset=domain, config_file_list=props)
    config['metrics'] = ['NDCG']
    config['topk'] = [10,50]
    config['data_path']=base_dir
    config['dataset']=domain
    config['device'] = 'cuda'


    init_seed(config['seed'], config['reproducibility'])

    # logger initialization
    init_logger(config)
    logger = getLogger()
    logger.info(config)

    # dataset creation
    dataset = create_dataset(config)
    logger.info(dataset)
    print('ORIGINAL DATASET:', len(dataset.inter_feat))

    # Prepare the data
    train_data, valid_data, test_data = data_preparation(config, dataset)


    # Filter users by history length in the test set
    filtered_test_dataset = filter_users_by_history_length(test_data.dataset, history_length=history_length)
    print(f'FILTERED TEST DATASET (history length = {history_length}):', len(filtered_test_dataset.inter_feat))

    if len(filtered_test_dataset.inter_feat) == 0:
        raise ValueError("No interactions left in test data after filtering. Please check your filter criteria.")

    # Create a sampler for the filtered dataset
    sampler = Sampler(config, filtered_test_dataset)

    # Create a new test_data with the filtered dataset
    filtered_test_data = FullSortEvalDataLoader(config, filtered_test_dataset, sampler)


    # model loading and initialization
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model_class(config, train_data.dataset).to(device)

    # trainer loading and initialization
    trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

    # model evaluation
    test_result = trainer.evaluate(filtered_test_data, load_best_model=False, model_file=model_file, show_progress=config['show_progress'])
    print("Test result: ", test_result)
    return test_result



model_name = 'UniSRec'
domains=['All_Beauty','Video_Games','Baby_Products']
models=['/kaggle/input/data-ablation/All_Beauty/UniSRec-Jul-22-2024_22-04-52.pth','/kaggle/input/data-ablation/Video_Games/checkpoint_unisrec/UniSRec-Jul-22-2024_23-04-56.pth','/kaggle/input/data-ablation/Baby_Products/checkpoint_unisrec/UniSRec-Jul-22-2024_22-11-11.pth']

base_dir = '/kaggle/input/data-ablation/Archive/'

all_results = {domain: {} for domain in domains}
for i,domain in enumerate(domains):
    ncdg_10=[]
    ncdg_50=[]
    for history in range(1,11):
        results = run(model_name,domain,base_dir+f'{domain}',models[i],history)
        ncdg_10.append(results['ndcg@10'])
        ncdg_50.append(results['ndcg@50'])
    all_results[domain]['ndcg@10']=ncdg_10
    all_results[domain]['ndcg@50']=ncdg_50



In [ ]:
#ST5 History Length Ablation

from tqdm import tqdm
import numpy as np
import os
import pandas as pd 
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer
import pandas as pd
import json
import torch
import argparse

device=('cuda' if torch.cuda.is_available() else 'cpu')
# model = SentenceTransformer('/kaggle/input/models/models/final_model', device=device,local_files_only=True)
#model = SentenceTransformer('sentence-transformers/sentence-t5-base', device=device)


# # List of domains
# domains = ['Baby_Products','Video_Games']

# models=['']

models=['sentence-transformers/sentence-t5-base','/kaggle/input/models/models/final_model'] #st5 only , st5 final model

domains=['All_Beauty','Video_Games','Baby_Products']

base_dir='/kaggle/input/recdata/data'

all_results = {domain: {model: {} for model in models} for domain in domains}


for model_name in models:
    print(f"Processing model: {model_name}")
    model = SentenceTransformer(model_name, device=device)
    tokenizer = model.tokenizer
    for domain in domains: 

        print(f"Processing domain: {domain}")

        
        #Load Data Maps
        data_maps_file = os.path.join(base_dir, domain, f"{domain}.data_maps")   
        with open(data_maps_file, 'r') as f:
            data_maps = json.load(f)

        for key,value in tqdm(data_maps['id2meta'].items()):
            new_value='<extra_id_1>'+value+'<extra_id_2>'
            #tokenize it
            tokenized_value=tokenizer.tokenize(new_value)
            if len(tokenized_value)<=255:
                data_maps['id2meta'][key]=new_value
            else:
                diff=len(tokenized_value)-255
                new_value='<extra_id_1>'+value[:-diff]+'<extra_id_2>'
                data_maps['id2meta'][key]=new_value

        # Load test data
        df=pd.read_csv(os.path.join(base_dir, domain, f"{domain}.test.csv"))
        df['count']=df['history_text'].apply(lambda x:x.count('<extra_id_1>'))
        count_values = df['count'].value_counts().sort_index()

        history_count_10=[]
        history_count_50=[]
        for i,count in enumerate(count_values[:7]):
            print(i+1,count)
            if (i+1>10):
                break
            temp=df[df['count']==i+1]

            queries = {}
            relevant_docs = {}
            for _, row in temp.iterrows():
                user_id = str(data_maps['user2id'][row['user_id']])
                item_id = str(data_maps['item2id'][row['parent_asin']])

                if user_id not in queries:
                    queries[user_id] = row['history_text']
                    relevant_docs[user_id] = []

                relevant_docs[user_id].append(item_id)

            # Create and run evaluator
            ir_evaluator = InformationRetrievalEvaluator(
                queries=queries,
                corpus=data_maps['id2meta'],
                relevant_docs=relevant_docs,
                ndcg_at_k=[1,10,50],
                precision_recall_at_k=[1,10,50],
                show_progress_bar=True,
                batch_size=32,
            )
            results = ir_evaluator(model)
            print(f"{domain}: NDCG@10:{results['cosine_ndcg@10']:.4f} NDGC@50:{results['cosine_ndcg@50']:.4f} Recall@10:{results['cosine_recall@10']:.4f} Recall@50:{results['cosine_recall@50']:.4f}")
            history_count_10.append(results['cosine_ndcg@10'])
            history_count_50.append(results['cosine_ndcg@50'])
        all_results[domain][model_name]['ndcg@10'] = history_count_10
        all_results[domain][model_name]['ndcg@50'] = history_count_50
        
            